<a href="https://colab.research.google.com/github/linzhe001/tutorial_notebooks/blob/main/CIFAR_10N_CNN_withNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloning repository of CIFAR-10H Annotation
Paper: Human uncertainty makes classification more robust (https://arxiv.org/pdf/1908.07086.pdf)

Label:  CIFAR10 [0: airplane, 1: automobile, 2: bird, 3: cat, 4: deer, 5: dog, 6: frog, 7: horse, 8: ship, 9: truck] <br>

<img src="https://miro.medium.com/max/1010/1*r8S5tF_6naagKOnlIcGXoQ.png" alt="alternatetext">




In [1]:
!git clone https://github.com/UCSC-REAL/cifar-10-100n.git
%cd cifar-10-100n

fatal: destination path 'cifar-10-100n' already exists and is not an empty directory.
/content/cifar-10-100n


# main script

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=12):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
## seed is used to ristrict randomness for the reproducibility purpose

parser = argparse.ArgumentParser(description='CIFAR-10H Training') ## create an argparse object
parser.add_argument('--lr', default=0.1, type=float, help='learning rate') ## add some augments
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--batch_size', default=1024, type=int, help='batch size')
parser.add_argument('--test_batch_size', default=2048, type=int, help='batch size')
parser.add_argument('--num_epoch', default=100, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=10, help='number classes')
args = parser.parse_args(args=[]) ## store theses data in args.Argument; can print by ``` args.lr ```
## notice ``` parser.parse_args ``` is necessary for this section, can not run ```parser.lr```

def train(model, trainloader, criterion, optimizer):
    model.train()
    for batch_idx, (inputs, targets, ad) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [8]:
print(args.lr)

0.1


# CIFAR-10H dataloader

In [ ]:
from PIL import Image
import numpy as np
import torchvision

class CIFAR10N(torchvision.datasets.CIFAR10):

    def __init__(self, root,  rand_number=0, train=False, transform=None, target_transform=None,
                 download=False, istrain=False):
        super(CIFAR10N, self).__init__(root, train, transform, target_transform, download)
        self.istrain = istrain
        self.transform = transform
        self.target_transform = target_transform
        ann_all = torch.load('./data/CIFAR-10_human.pt')
        ann_ = np.array([ann_all['random_label1'], ann_all['random_label2'], ann_all['random_label3']]).transpose(1,0)
        self.ad = np.zeros((50000, 10))
        for idx, ann_per_img in enumerate(ann_):
            for ann_per_rater in ann_per_img:
                self.ad[idx, ann_per_rater] += 1

    def __getitem__(self, index: int):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        if self.istrain:
            ad = self.ad[index]
            return img, target, ad
        else:
            return img, target


# Run script

In [ ]:
seed_everything()
mean_cifar10, std_cifar10 = (0.5071, 0.4866, 0.4409), (0.2009, 0.1984, 0.2023)
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(), transforms.ToTensor(),
            transforms.Normalize(mean_cifar10, std_cifar10), ])
transform_test = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize(mean_cifar10, std_cifar10),])

train_dataset = CIFAR10N(root='./data', train=True, download=True, transform=transform_train, istrain=True)
test_dataset = CIFAR10N(root='./data', train=False, download=True, transform=transform_test, istrain=False)
#test_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_test)
print('train samples:',len(train_dataset), 'test samples:',len(test_dataset))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=2)

model = models.resnet34(pretrained=True).to(device)
model.fc = nn.Linear(model.fc.in_features, args.num_classes)
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, nesterov=False, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()

best_epoch, best_acc = 0.0, 0
for epoch in range(args.num_epoch):
    train(model, train_loader, criterion, optimizer)
    accuracy = test(model, test_loader)
    if accuracy > best_acc:
        patience = 0
        best_acc = accuracy
        best_epoch = epoch
        best_model = copy.deepcopy(model)
        torch.save(best_model.state_dict(), 'best_model_cifar10h.pth.tar')
    print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
            epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))

100%|██████████| 170498071/170498071 [00:13<00:00, 12956801.73it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


<ipython-input-3-8fdc7fd4e1ad>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ann_all = torch.load('./data/CIFAR-10_human.pt')


Files already downloaded and verified
train samples: 50000 test samples: 10000


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 145MB/s]


epoch: 0  acc: 0.6230  best epoch: 0  best acc: 0.6230
epoch: 1  acc: 0.7623  best epoch: 1  best acc: 0.7623
epoch: 2  acc: 0.7848  best epoch: 2  best acc: 0.7848
epoch: 3  acc: 0.8146  best epoch: 3  best acc: 0.8146
epoch: 4  acc: 0.8140  best epoch: 3  best acc: 0.8146
epoch: 5  acc: 0.7897  best epoch: 3  best acc: 0.8146


In [ ]:
import tensorflow_probability as tfp

def evaluation_all(model, testloader):
    model.eval()
    logits_list = []
    labels_list = []
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            logits_list.append(outputs)
            labels_list.append(targets)

        logits = torch.cat(logits_list).cpu().numpy()
        labels = torch.cat(labels_list).cpu().numpy()
    return correct / total, logits, labels

model.load_state_dict(torch.load('best_model_cifar10h.pth.tar'))
acc, logits_tf, labels_tf = evaluation_all(model, test_loader)
ece = tfp.stats.expected_calibration_error(args.num_classes, logits=logits_tf, labels_true=labels_tf, labels_predicted=np.argmax(logits_tf,1))
print("Acc:{:.4f}, ECE:{:.4f}".format(acc, np.array(ece)))

Acc:0.8076, ECE:0.0799
